## Capstone project

## Read in the new sample file from Radhika with only 10 HCPCS codes

In [2]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
plt.style.use('ggplot')
%matplotlib inline

In [3]:
hcpcs10 = pd.read_csv('top10_hcpcs.csv', index_col=0)

# hcpcs10 = pd.read_csv('top10_hcpcs.csv', index_col=0)
# hcpcs10 = hcpcs10.reset_index()

C:\Users\aparn\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\aparn\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
pd.set_option("display.max_columns",500)
hcpcs10.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt
6,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00
7,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99232,"Subsequent hospital inpatient care, typically ...",N,627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94
8,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99233,"Subsequent hospital inpatient care, typically ...",N,207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88
10,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99291,Critical care delivery critically ill or injur...,N,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00
39,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,99213,Established patient office or other outpatient...,N,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98


In [5]:
hcpcs10.dtypes

npi                                   int64
nppes_provider_last_org_name         object
nppes_provider_first_name            object
nppes_provider_mi                    object
nppes_credentials                    object
nppes_provider_gender                object
nppes_entity_code                    object
nppes_provider_street1               object
nppes_provider_street2               object
nppes_provider_city                  object
nppes_provider_zip                   object
nppes_provider_state                 object
nppes_provider_country               object
provider_type                        object
medicare_participation_indicator     object
place_of_service                     object
hcpcs_code                           object
hcpcs_description                    object
hcpcs_drug_indicator                 object
line_srvc_cnt                       float64
bene_unique_cnt                       int64
bene_day_srvc_cnt                     int64
average_Medicare_allowed_amt    

In [9]:
hcpcs10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520985 entries, 6 to 9847441
Data columns (total 28 columns):
npi                                 1520985 non-null int64
nppes_provider_last_org_name        1520959 non-null object
nppes_provider_first_name           1511915 non-null object
nppes_provider_mi                   1075160 non-null object
nppes_credentials                   1455318 non-null object
nppes_provider_gender               1511939 non-null object
nppes_entity_code                   1520985 non-null object
nppes_provider_street1              1520983 non-null object
nppes_provider_street2              632192 non-null object
nppes_provider_city                 1520985 non-null object
nppes_provider_zip                  1520985 non-null object
nppes_provider_state                1520985 non-null object
nppes_provider_country              1520985 non-null object
provider_type                       1520985 non-null object
medicare_participation_indicator    1520985 non-n

In [6]:
hcpcs10.columns[hcpcs10.isnull().any()].tolist()

['nppes_provider_last_org_name',
 'nppes_provider_first_name',
 'nppes_provider_mi',
 'nppes_credentials',
 'nppes_provider_gender',
 'nppes_provider_street1',
 'nppes_provider_street2']

In [7]:
## Get a count of the missings by column.

len(hcpcs10.index) - hcpcs10.count()

npi                                      0
nppes_provider_last_org_name            26
nppes_provider_first_name             9070
nppes_provider_mi                   445825
nppes_credentials                    65667
nppes_provider_gender                 9046
nppes_entity_code                        0
nppes_provider_street1                   2
nppes_provider_street2              888793
nppes_provider_city                      0
nppes_provider_zip                       0
nppes_provider_state                     0
nppes_provider_country                   0
provider_type                            0
medicare_participation_indicator         0
place_of_service                         0
hcpcs_code                               0
hcpcs_description                        0
hcpcs_drug_indicator                     0
line_srvc_cnt                            0
bene_unique_cnt                          0
bene_day_srvc_cnt                        0
average_Medicare_allowed_amt             0
average_sub

In [49]:
hcpcs10.loc[:,['hcpcs_code','hcpcs_description','line_srvc_cnt','average_submitted_chrg_amt','average_Medicare_payment_amt']]

,hcpcs_code,hcpcs_description,line_srvc_cnt,average_submitted_chrg_amt,average_Medicare_payment_amt
6,99223,"Initial hospital inpatient care, typically 70 ...",114.0,923.991228,160.377193
7,99232,"Subsequent hospital inpatient care, typically ...",627.0,326.245614,56.366730
8,99233,"Subsequent hospital inpatient care, typically ...",207.0,461.072464,82.820676
10,99291,Critical care delivery critically ill or injur...,36.0,1400.000000,175.750000
39,99213,Established patient office or other outpatient...,129.0,109.000000,50.868062
40,99214,Established patient office or other outpatient...,24.0,115.000000,58.658333
41,99214,Established patient office or other outpatient...,330.0,159.000000,73.218515
48,99223,"Initial hospital inpatient care, typically 70 ...",74.0,243.391892,149.435000
51,99232,"Subsequent hospital inpatient care, typically ...",241.0,150.000000,53.631328
52,99233,"Subsequent hospital inpatient care, typically ...",76.0,175.000000,78.250000


**Clean up the Zip Code variable**

In [30]:
# to get the first 5 digits of zip code, first convert the zip code var to string. 

hcpcs10["nppes_provider_zip"]= hcpcs10["nppes_provider_zip"].astype(str) 

In [31]:
# Now use string slicing to get the first five digits.

hcpcs10['prov_zip'] = hcpcs10.nppes_provider_zip.str.slice(0,5,1)

In [32]:
hcpcs10.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,prov_zip
6,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,21502
7,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99232,"Subsequent hospital inpatient care, typically ...",N,627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,21502
8,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99233,"Subsequent hospital inpatient care, typically ...",N,207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,21502
10,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99291,Critical care delivery critically ill or injur...,N,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,21502
39,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,99213,Established patient office or other outpatient...,N,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,43623


**Divide the file into 2 DFs -- one for individuals and the other for organizations**

In [52]:
hcpcs10['nppes_entity_code'].nunique()

2

In [53]:
hcpcs10['nppes_entity_code'].value_counts()

I    1511939
O       9046
Name: nppes_entity_code, dtype: int64

**Get the individual level file and check it**

In [56]:
hcpcs10_ind = hcpcs10[hcpcs10['nppes_entity_code']=='I']

In [57]:
type(hcpcs10_ind)

pandas.core.frame.DataFrame

In [58]:
hcpcs10_ind.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,prov_zip
6,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,21502
7,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99232,"Subsequent hospital inpatient care, typically ...",N,627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,21502
8,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99233,"Subsequent hospital inpatient care, typically ...",N,207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,21502
10,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99291,Critical care delivery critically ill or injur...,N,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,21502
39,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,99213,Established patient office or other outpatient...,N,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,43623


In [73]:
# Obtain a list of cols with missings

hcpcs10_ind.columns[hcpcs10_ind.isnull().any()].tolist()

['nppes_provider_last_org_name',
 'nppes_provider_first_name',
 'nppes_provider_mi',
 'nppes_credentials',
 'nppes_provider_street1',
 'nppes_provider_street2']

In [61]:
# Get the count of missings by columns in df

len(hcpcs10_ind.index) - hcpcs10_ind.count()

npi                                      0
nppes_provider_last_org_name            26
nppes_provider_first_name               24
nppes_provider_mi                   436779
nppes_credentials                    56621
nppes_provider_gender                    0
nppes_entity_code                        0
nppes_provider_street1                   2
nppes_provider_street2              880777
nppes_provider_city                      0
nppes_provider_zip                       0
nppes_provider_state                     0
nppes_provider_country                   0
provider_type                            0
medicare_participation_indicator         0
place_of_service                         0
hcpcs_code                               0
hcpcs_description                        0
hcpcs_drug_indicator                     0
line_srvc_cnt                            0
bene_unique_cnt                          0
bene_day_srvc_cnt                        0
average_Medicare_allowed_amt             0
average_sub

In [68]:
## View the provider last name and first name
## Using head as shown in next box is more efficient.

hcpcs10_ind.loc[:,['npi','nppes_provider_last_org_name','nppes_provider_first_name']]

In [67]:
hcpcs10_ind[['npi','nppes_provider_last_org_name','nppes_provider_first_name']].head(20)

,npi,nppes_provider_last_org_name,nppes_provider_first_name
6,1003000126,ENKESHAFI,ARDALAN
7,1003000126,ENKESHAFI,ARDALAN
8,1003000126,ENKESHAFI,ARDALAN
10,1003000126,ENKESHAFI,ARDALAN
39,1003000142,KHALIL,RASHID
40,1003000142,KHALIL,RASHID
41,1003000142,KHALIL,RASHID
48,1003000407,GIRARDI,DAVID
51,1003000407,GIRARDI,DAVID
52,1003000407,GIRARDI,DAVID


In [70]:
hcpcs10_ind['nppes_provider_last_org_name'].value_counts()

PATEL               9159
SMITH               7574
LEE                 5437
JOHNSON             5431
MILLER              4762
SHAH                4186
BROWN               4180
WILLIAMS            3908
JONES               3847
KHAN                3581
KIM                 3414
NGUYEN              3315
SINGH               3010
DAVIS               3008
ANDERSON            3005
THOMAS              2863
WILSON              2611
MARTIN              2476
MOORE               2319
TAYLOR              2167
GUPTA               2091
THOMPSON            2090
CHEN                2033
HARRIS              1935
WHITE               1928
YOUNG               1883
SHARMA              1863
AHMED               1854
REDDY               1850
LEWIS               1813
                    ... 
FARD                   1
TRUTNAU                1
STRICH                 1
VAHI                   1
MENZOIAN               1
AJEENA                 1
SUAREZ-MEDEROS         1
EVELSON                1
BOLOURI                1


**Create the organization level dataframe and check it**

In [71]:
## Subset the organization entities to examine them closely, independent of individuals.

hcpcs10_org = hcpcs10[hcpcs10['nppes_entity_code']=='O']

In [74]:
hcpcs10_org.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,prov_zip
844,1003006180,"MEDICWEST AMBULANCE, INC.",NaN,NaN,NaN,NaN,O,9 W DELHI AVE,NaN,NORTH LAS VEGAS,890327836,NV,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,11857.0,7355,11824,441.111760,1020.810443,330.997026,322.372015,1.210375e+07,3.924632e+06,89032
2692,1003015355,MURRAYVILLE WOODSON EMERGENCY AMBULANCE SERVIC...,NaN,NaN,NaN,NaN,O,1930 STATE HIGHWAY 267,NaN,MURRAYVILLE,62668,IL,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,18.0,15,18,419.096667,733.333333,328.568889,335.160000,1.320000e+04,5.914240e+03,62668
2938,1003015843,"NORTHBAY PHYSICIAN'S SURGERY CENTER, L.L.C.",NaN,NaN,NaN,NaN,O,1006 NUT TREE ROAD,NaN,VACAVILLE,95687,CA,US,Ambulatory Surgical Center,Y,F,66984,Removal of cataract with insertion of lens,N,318.0,206,318,1317.601509,9253.144654,1023.232421,747.722201,2.942500e+06,3.253879e+05,95687
6245,1003035213,"EAST HILLS AMBULANCE, INC",NaN,NaN,NaN,NaN,O,3111 ELTON RD,NaN,JOHNSTOWN,159042731,PA,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,469.0,327,469,410.967463,947.547974,313.595373,326.508316,4.444000e+05,1.470762e+05,15904
8493,1003047390,CITY OF WESTERVILLE,NaN,NaN,NaN,NaN,O,400 W MAIN ST,NaN,WESTERVILLE,430811436,OH,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,672.0,547,672,406.070000,815.120000,317.138393,334.723378,5.477606e+05,2.131170e+05,43081


In [77]:
hcpcs10_org.shape

(9046, 29)

In [75]:
# list of columns that have missing values

hcpcs10_org.columns[hcpcs10_org.isnull().any()].tolist()

['nppes_provider_first_name',
 'nppes_provider_mi',
 'nppes_credentials',
 'nppes_provider_gender',
 'nppes_provider_street2']

In [76]:
# number of missing values by column

len(hcpcs10_org.index) - hcpcs10_org.count()

npi                                    0
nppes_provider_last_org_name           0
nppes_provider_first_name           9046
nppes_provider_mi                   9046
nppes_credentials                   9046
nppes_provider_gender               9046
nppes_entity_code                      0
nppes_provider_street1                 0
nppes_provider_street2              8016
nppes_provider_city                    0
nppes_provider_zip                     0
nppes_provider_state                   0
nppes_provider_country                 0
provider_type                          0
medicare_participation_indicator       0
place_of_service                       0
hcpcs_code                             0
hcpcs_description                      0
hcpcs_drug_indicator                   0
line_srvc_cnt                          0
bene_unique_cnt                        0
bene_day_srvc_cnt                      0
average_Medicare_allowed_amt           0
average_submitted_chrg_amt             0
average_Medicare

In [81]:
hcpcs10_org['nppes_provider_last_org_name'].unique().tolist()

['MEDICWEST AMBULANCE, INC.',
 'MURRAYVILLE WOODSON EMERGENCY AMBULANCE SERVICE, INC.',
 "NORTHBAY PHYSICIAN'S SURGERY CENTER, L.L.C.",
 'EAST HILLS AMBULANCE, INC',
 'CITY OF WESTERVILLE',
 'RICHMOND COUNTY BOARD OF SUPERVISORS',
 'ADVANCED SURGICAL CARE OF BATON ROUGE,LLC',
 'CHESTER TOWNSHIP TRUSTEES',
 'STONE COUNTY HOSPITAL, INC',
 'OHIO AMBULANCE SOLUTIONS, LLC',
 'WASC LLC',
 'DHEW IND HLTH SV HLTH SVS & MNTL HLTH ADM',
 'CITY OF SHEBOYGAN',
 'BEAUMONT MEDICAL TRANSPORTATION SERVICES, INC',
 'EMS DIRECT LLC',
 'FREDERICKTOWN COMMUNITY JOINT EMERGENCY AMBULANCE DISTRICT',
 'VILLAGE OF LORDSTOWN',
 'MOUNTAIN WEST SURGERY CENTER, LLC',
 'DELTA RESPONSE TEAM',
 'KING WILLIAM COUNTY',
 'CITY OF WARREN',
 'HEALTHFLEET AMBULANCE, INC.',
 'CANDESCENT EYE HEALTH SURGICENTER, LLC',
 'CITY OF BELLA VISTA AMBULANCE SERVICE',
 'HOCKING TOWNSHIP',
 'HARTFORD VOLUNTEER FIRE DEPARTMENT INC',
 'GREAT LAKES SURGERY CENTER LLC',
 'EASTON EMERGENCY SQUAD',
 'SURGERY CENTER OF MIDWEST CITY, LLC',
 '

In [86]:
hcpcs10_org['nppes_provider_last_org_name'].value_counts()

AMERICAN MEDICAL RESPONSE AMBULANCE SERVICE INC             17
PACIFIC CATARACT AND LASER INSTITUTE INC PC                 16
SOUTHWESTERN EYE CENTER LTD                                 13
LIFEGUARD AMBULANCE SERVICE LLC                             13
AMERICAN MEDICAL RESPONSE WEST                              12
HEARTLAND EMS INC                                            8
BARNET DULANEY PERKINS EYE CENTER, PLLC                      8
MED-TRANS CORPORATION                                        8
AMERICAN MEDICAL RESPONSE OF COLORADO INC                    5
AMERICAN MEDICAL RESPONSE NORTHWEST INC                      5
NATIONAL PARK SERVICE                                        5
RURAL METRO CORPORATION                                      4
PCI GROUP LLC                                                4
CITY OF FRANKLIN                                             4
TOWN OF WINDHAM                                              4
WASHINGTON TOWNSHIP                                    

**Make changes to the org name string**

In [106]:
## Convert Org last name to lower case

hcpcs10_org['nppes_provider_last_org_name'] = hcpcs10_org['nppes_provider_last_org_name'].apply(lambda x: x.lower())

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [117]:
## Remove punctuations etc from org name
## import string -- for later

hcpcs10_org['nppes_provider_last_org_name'] = hcpcs10_org['nppes_provider_last_org_name'].str.replace(r'[^\w\s]+', '')

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [120]:
hcpcs10_org.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,prov_zip
844,1003006180,medicwest ambulance inc,NaN,NaN,NaN,NaN,O,9 W DELHI AVE,NaN,NORTH LAS VEGAS,890327836,NV,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,11857.0,7355,11824,441.111760,1020.810443,330.997026,322.372015,1.210375e+07,3.924632e+06,89032
2692,1003015355,murrayville woodson emergency ambulance servic...,NaN,NaN,NaN,NaN,O,1930 STATE HIGHWAY 267,NaN,MURRAYVILLE,62668,IL,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,18.0,15,18,419.096667,733.333333,328.568889,335.160000,1.320000e+04,5.914240e+03,62668
2938,1003015843,northbay physicians surgery center llc,NaN,NaN,NaN,NaN,O,1006 NUT TREE ROAD,NaN,VACAVILLE,95687,CA,US,Ambulatory Surgical Center,Y,F,66984,Removal of cataract with insertion of lens,N,318.0,206,318,1317.601509,9253.144654,1023.232421,747.722201,2.942500e+06,3.253879e+05,95687
6245,1003035213,east hills ambulance inc,NaN,NaN,NaN,NaN,O,3111 ELTON RD,NaN,JOHNSTOWN,159042731,PA,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,469.0,327,469,410.967463,947.547974,313.595373,326.508316,4.444000e+05,1.470762e+05,15904
8493,1003047390,city of westerville,NaN,NaN,NaN,NaN,O,400 W MAIN ST,NaN,WESTERVILLE,430811436,OH,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,672.0,547,672,406.070000,815.120000,317.138393,334.723378,5.477606e+05,2.131170e+05,43081


In [119]:
hcpcs10_org['nppes_provider_last_org_name'].unique().tolist()

['medicwest ambulance inc',
 'murrayville woodson emergency ambulance service inc',
 'northbay physicians surgery center llc',
 'east hills ambulance inc',
 'city of westerville',
 'richmond county board of supervisors',
 'advanced surgical care of baton rougellc',
 'chester township trustees',
 'stone county hospital inc',
 'ohio ambulance solutions llc',
 'wasc llc',
 'dhew ind hlth sv hlth svs  mntl hlth adm',
 'city of sheboygan',
 'beaumont medical transportation services inc',
 'ems direct llc',
 'fredericktown community joint emergency ambulance district',
 'village of lordstown',
 'mountain west surgery center llc',
 'delta response team',
 'king william county',
 'city of warren',
 'healthfleet ambulance inc',
 'candescent eye health surgicenter llc',
 'city of bella vista ambulance service',
 'hocking township',
 'hartford volunteer fire department inc',
 'great lakes surgery center llc',
 'easton emergency squad',
 'surgery center of midwest city llc',
 'minnesota eye instit

In [163]:
temporg = hcpcs10_org['nppes_provider_last_org_name'].str.split(' ', expand=True).rename(columns = lambda x: "ORG"+str(x+1))

In [166]:
type(temporg)
temporg.head()
temporg.shape

(9046, 14)

In [167]:
hcpcs10_org = hcpcs10_org.join(temporg)

In [169]:
hcpcs10_org.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,prov_zip,ORG1,ORG2,ORG3,ORG4,ORG5,ORG6,ORG7,ORG8,ORG9,ORG10,ORG11,ORG12,ORG13,ORG14
844,1003006180,medicwest ambulance inc,NaN,NaN,NaN,NaN,O,9 W DELHI AVE,NaN,NORTH LAS VEGAS,890327836,NV,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,11857.0,7355,11824,441.111760,1020.810443,330.997026,322.372015,1.210375e+07,3.924632e+06,89032,medicwest,ambulance,inc,None,None,None,None,None,None,None,None,None,None,None
2692,1003015355,murrayville woodson emergency ambulance servic...,NaN,NaN,NaN,NaN,O,1930 STATE HIGHWAY 267,NaN,MURRAYVILLE,62668,IL,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,18.0,15,18,419.096667,733.333333,328.568889,335.160000,1.320000e+04,5.914240e+03,62668,murrayville,woodson,emergency,ambulance,service,inc,None,None,None,None,None,None,None,None
2938,1003015843,northbay physicians surgery center llc,NaN,NaN,NaN,NaN,O,1006 NUT TREE ROAD,NaN,VACAVILLE,95687,CA,US,Ambulatory Surgical Center,Y,F,66984,Removal of cataract with insertion of lens,N,318.0,206,318,1317.601509,9253.144654,1023.232421,747.722201,2.942500e+06,3.253879e+05,95687,northbay,physicians,surgery,center,llc,None,None,None,None,None,None,None,None,None
6245,1003035213,east hills ambulance inc,NaN,NaN,NaN,NaN,O,3111 ELTON RD,NaN,JOHNSTOWN,159042731,PA,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,469.0,327,469,410.967463,947.547974,313.595373,326.508316,4.444000e+05,1.470762e+05,15904,east,hills,ambulance,inc,None,None,None,None,None,None,None,None,None,None
8493,1003047390,city of westerville,NaN,NaN,NaN,NaN,O,400 W MAIN ST,NaN,WESTERVILLE,430811436,OH,US,Ambulance Service Provider,Y,F,A0427,"Ambulance service, advanced life support, emer...",N,672.0,547,672,406.070000,815.120000,317.138393,334.723378,5.477606e+05,2.131170e+05,43081,city,of,westerville,None,None,None,None,None,None,None,None,None,None,None


**Work with HCPCS10 address fields**

In [104]:
hcpcs10_org['nppes_provider_street1'].unique().tolist()

['9 W DELHI AVE',
 '1930 STATE HIGHWAY 267',
 '1006 NUT TREE ROAD',
 '3111 ELTON RD',
 '400 W MAIN ST',
 '101 COURT CIRCLE',
 '7310 PERKINS ROAD',
 '5580 ST RT 380',
 '1434 CENTRAL AVE E',
 '26420 LAKELAND BLVD',
 '3373 COMMERCE PARKWAY',
 '4212 NORTH 16TH STREET',
 '1326 N 25TH ST',
 '25400 W 8 MILE RD',
 '316-B SOUTH 6TH STREET',
 '139 COLUMBUS RD',
 '1595 SALT SPRINGS RD',
 '1551 SOUTH RENAISSANCE TOWNE DR',
 '698 JONES ST',
 '180 HORSE LANDING ROAD #4',
 '1 CITY SQ',
 '235 NEW YORK DR',
 '51 STATE ROAD',
 '103 TOWN CTR W',
 '1175 CINCINNATI ZANESVILLE RD SW',
 '180 SMITH STREET',
 '2848 NILES RD',
 '908 PACKER ST',
 '8121 NATIONAL AVE',
 '3401 S BROADWAY',
 '29 ELM ST',
 '225 PARK ST',
 '200 S MIAMI AVE',
 '402 MERCER ST',
 '67 E CLARK ST',
 '319 STANFORD ST',
 '110 N MAIN ST',
 '606 W POTTER AVE',
 '1450 ROSS CLARK CIR',
 '100 S WILLIAMS ST',
 '2200 HUNTINGTON DR',
 '895 MAIN ST',
 '100 E CHARLOTTE ST',
 '13210 GOODLAND ST',
 '6591 ORANGE DR',
 '1000 BROADWAY DRIVE',
 '333 W. LAKE

In [105]:
hcpcs10_org['nppes_provider_street2'].unique().tolist()

[nan,
 'SUITE 1',
 'PHOENIX INDIAN MEDICAL CENTER AMBULATORY SURGICAL CENT',
 'SUITE 20',
 'SUITE 200',
 'SUITE B',
 'SUITE 108',
 'SUITE 4',
 'SUITE 46',
 'BOX 538',
 '902 US HWY 41 NW',
 'SUITE 800',
 'BOX 205',
 'SUITE 100',
 'STE. U',
 'SUITE 120',
 'SUITE 104',
 'HC 63 - BOX 133',
 '1 KENNEDY PLAZA',
 'SUITE 110',
 'STE 950',
 '65 AVENUE',
 'SUITE 11B',
 'MAIN FIRE STATION',
 'SUITE 400',
 'SUITE 10E',
 '14 WILSON DRIVE',
 'STE 400',
 'STATION ONE',
 'SUITE 330',
 'SUITE 124',
 'SUITE D',
 'SUITE 150',
 'STE 102',
 'ROOM 108',
 'SUITE 130',
 'SUITE 101',
 'SUITE 1515',
 'STE 100',
 'STE 204',
 'SUITE 2',
 'BLDG 7',
 'SUITE 5',
 'SUITE 160',
 'PULASKI COUNTY AMBULANCE',
 'SUITE 107',
 'SPACE #225',
 'HASKELL COUNTY COURT HOUSE',
 'SUITE 500',
 'SUITE 103',
 'SUITE 180',
 'SUITE 2000',
 'BYPASS N',
 'SUITE 118',
 'STE 2',
 'SUITE 315',
 'SUITE 102',
 'SUITE C 2020',
 'SUITE A',
 'SUITE S4',
 'STE 202A',
 '#11',
 'STE A',
 'STE 303',
 '10TH FLOOR, SUITE 1000',
 'SUITE C',
 'SUITE 105

**Make changes to the street1 variable to enable easier match with LEIE**

In [170]:
## Convert the address field to lower case

hcpcs10_org['nppes_provider_street1']= hcpcs10_org['nppes_provider_street1'].apply(lambda x: x.lower())

In [171]:
## Remove punctuations from address field

hcpcs10_org['nppes_provider_street1'] = hcpcs10_org['nppes_provider_street1'].str.replace(r'[^\w\s]+', '')

In [173]:
hcpcs10_org.head()

hcpcs10_org['nppes_provider_street1'].unique().tolist()

['9 w delhi ave',
 '1930 state highway 267',
 '1006 nut tree road',
 '3111 elton rd',
 '400 w main st',
 '101 court circle',
 '7310 perkins road',
 '5580 st rt 380',
 '1434 central ave e',
 '26420 lakeland blvd',
 '3373 commerce parkway',
 '4212 north 16th street',
 '1326 n 25th st',
 '25400 w 8 mile rd',
 '316b south 6th street',
 '139 columbus rd',
 '1595 salt springs rd',
 '1551 south renaissance towne dr',
 '698 jones st',
 '180 horse landing road 4',
 '1 city sq',
 '235 new york dr',
 '51 state road',
 '103 town ctr w',
 '1175 cincinnati zanesville rd sw',
 '180 smith street',
 '2848 niles rd',
 '908 packer st',
 '8121 national ave',
 '3401 s broadway',
 '29 elm st',
 '225 park st',
 '200 s miami ave',
 '402 mercer st',
 '67 e clark st',
 '319 stanford st',
 '110 n main st',
 '606 w potter ave',
 '1450 ross clark cir',
 '100 s williams st',
 '2200 huntington dr',
 '895 main st',
 '100 e charlotte st',
 '13210 goodland st',
 '6591 orange dr',
 '1000 broadway drive',
 '333 w lake st

In [174]:
hcpcs10_org['nppes_provider_street1'].value_counts()

5005 sunset blvd            6
617 w main st               4
123 s grace st              4
395 w lake st               4
300 main st                 4
668 hernandez carrion       4
8400 183rd pl               3
1000 battles st             3
10361 spartan dr            3
130 n main st               3
9 main st ste 2k            3
800 main st                 3
100 e main st               3
59 main st                  3
main street                 3
237 oakland dr              3
1 main st                   3
25 main st                  2
284 main st                 2
117 south fayette street    2
110 n main st               2
11606 nicholas st           2
290 armistice blvd          2
63 bo pete manor rd         2
1125 main st                2
1180 n indian canyon dr     2
128 bucksport rd            2
109 n main st               2
2593 w 13th st              2
70 city hall ave            2
                           ..
200 2nd st ne               1
116 michigan ave            1
8214 richa

## Read in the LEIE file from Sumi here

In [37]:
leie_m = pd.read_csv('LEIE_ALL_17_19.csv', dtype={'ZIP': object})

#leie_m = pd.read_csv('LEIE_ALL_17_19', index_col=0)

In [38]:
leie_m.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,EXCLYEAR
0,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,2018-04-19,0,0,NaN,2018
1,NaN,NaN,,"716 TRANSPORTATION, INC",OTHER BUSINESS,TRANSPORTATION CO,NaN,0,NaN,540 VINE LANE,BUFFALO,NY,14228,1128a1,2019-06-20,0,0,NaN,2019
2,NaN,NaN,,A & H VITAMINS SUPPLY CORP,OTHER BUSINESS,PHARMACY,NaN,0,NaN,592 PALISADE AVENUE,TEANECK,NJ,7666,1128a1,2018-04-19,0,0,NaN,2018
3,NaN,NaN,,"A & Y MEDICAL SUPPLY, INC",DME COMPANY,DME - GENERAL,NaN,1942476080,NaN,"6310 108TH STREET, APT 6J",FOREST HILLS,NY,11375,1128b8,2017-05-18,0,0,NaN,2017
4,NaN,NaN,,"A FAIR DEAL PHARMACY, INC",OTHER BUSINESS,PHARMACY,NaN,1891731758,NaN,"C/O P O BOX 329014, #69709-05",BROOKLYN,NY,11232,1128b8,2017-05-18,0,0,NaN,2017


In [39]:
leie_m.dtypes

LASTNAME       object
FIRSTNAME      object
MIDNAME        object
BUSNAME        object
GENERAL        object
SPECIALTY      object
UPIN           object
NPI             int64
DOB           float64
ADDRESS        object
CITY           object
STATE          object
ZIP            object
EXCLTYPE       object
EXCLDATE       object
REINDATE        int64
WAIVERDATE      int64
WVRSTATE      float64
EXCLYEAR        int64
dtype: object

**Fix the Zip code variable**

In [41]:
## Check the length of the zip code var. Create a variable that stores the length of each zip code and freq it.

leie_m['zip_len']=leie_m['ZIP'].str.len()

In [50]:
# Obtain the frequency counts

leie_m['zip_len'].value_counts()

5    7120
4     540
3      22
1       5
Name: zip_len, dtype: int64

In [44]:
## Since there are several that are not 5 digit codes, add leading zeros to them. NJ codes have leading zeros.
## This works for the 4 digit codes
## There are 27 that are 3 or 1 digit long. These are problems that we can drop if necessary.

leie_m['zip_new']= leie_m['ZIP'].apply(lambda x: x.zfill(5))

In [45]:
leie_m.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,EXCLYEAR,zip_len,zip_new
0,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,2018-04-19,0,0,NaN,2018,5,10468
1,NaN,NaN,,"716 TRANSPORTATION, INC",OTHER BUSINESS,TRANSPORTATION CO,NaN,0,NaN,540 VINE LANE,BUFFALO,NY,14228,1128a1,2019-06-20,0,0,NaN,2019,5,14228
2,NaN,NaN,,A & H VITAMINS SUPPLY CORP,OTHER BUSINESS,PHARMACY,NaN,0,NaN,592 PALISADE AVENUE,TEANECK,NJ,7666,1128a1,2018-04-19,0,0,NaN,2018,4,07666
3,NaN,NaN,,"A & Y MEDICAL SUPPLY, INC",DME COMPANY,DME - GENERAL,NaN,1942476080,NaN,"6310 108TH STREET, APT 6J",FOREST HILLS,NY,11375,1128b8,2017-05-18,0,0,NaN,2017,5,11375
4,NaN,NaN,,"A FAIR DEAL PHARMACY, INC",OTHER BUSINESS,PHARMACY,NaN,1891731758,NaN,"C/O P O BOX 329014, #69709-05",BROOKLYN,NY,11232,1128b8,2017-05-18,0,0,NaN,2017,5,11232


In [48]:
leie_m.shape

(7687, 21)

In [87]:
leie_m.columns.tolist()

['LASTNAME',
 'FIRSTNAME',
 'MIDNAME',
 'BUSNAME',
 'GENERAL',
 'SPECIALTY',
 'UPIN',
 'NPI',
 'DOB',
 'ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'EXCLTYPE',
 'EXCLDATE',
 'REINDATE',
 'WAIVERDATE',
 'WVRSTATE',
 'EXCLYEAR',
 'zip_len',
 'zip_new']

**Create two new DFs -- one for Individuals and another for organizations**

**First create a DF for organizations**

In [150]:
## Subset the organizations to examine them more closely without individuals

leie_org = leie_m[leie_m['BUSNAME'].notnull()]

In [151]:
type(leie_org)

pandas.core.frame.DataFrame

In [152]:
leie_org.shape

(143, 21)

In [153]:
## Get a list of columns with missing values
## Note that this is slightly misleading, since missing NPI is set to zero

leie_org.columns[leie_org.isnull().any()].tolist()

['LASTNAME', 'FIRSTNAME', 'UPIN', 'DOB', 'WVRSTATE']

In [154]:
## Get a count of the missing values by column

len(leie_org.index) - leie_org.count()

LASTNAME      143
FIRSTNAME     143
MIDNAME         0
BUSNAME         0
GENERAL         0
SPECIALTY       0
UPIN          143
NPI             0
DOB           143
ADDRESS         0
CITY            0
STATE           0
ZIP             0
EXCLTYPE        0
EXCLDATE        0
REINDATE        0
WAIVERDATE      0
WVRSTATE      143
EXCLYEAR        0
zip_len         0
zip_new         0
dtype: int64

In [155]:
leie_org['BUSNAME'].unique().tolist()

['184TH STREET PHARMACY CORP',
 '716 TRANSPORTATION, INC',
 'A & H VITAMINS SUPPLY CORP',
 'A & Y MEDICAL SUPPLY, INC',
 'A FAIR DEAL PHARMACY, INC',
 'A SERVICE CAB CO, INC',
 'AAA PLUS HOME HEALTH CARE, LLC',
 'ACACIA MENTAL HEALTH CLINIC, L',
 'ACE CLINIQUE OF MEDICINE, LLC',
 'ADEL PHYSICAL THERAPY, PC',
 'ALBERT ADES, M D, P A',
 'ALLERTON PARK PHARMACY, INC',
 'AMBAMA CLINIC, P C',
 'ANDAS PHARMACEUTICAL SUPPLY CO',
 'AOC-DME CORP',
 'ARMS OF GRACE HUMANITARIAN SER',
 'ASTRAMED PHYSICIAN PC',
 'AUTUMN HEALTH CARE OF ZANESVIL',
 'AVENUE T CHIROPRACTIC PC',
 'BECAUSE WE CARE, LLC',
 'BELL GARDENS PHARMACY',
 'BENNETT SURGICAL SUPPLY, INC',
 'BEST CARE MEDICAL MANAGEMENT,',
 'BESTCARE LABORATORY SERVICES,',
 'BIODIAGNOSTIC LABORATORY SERVI',
 'BOLA ELEMUREN, MD PROFESSIONAL',
 'BRIGHT RX, INC',
 'BROOKLYN COMPREHENSIVE CARE ME',
 'CALIFORIA IMAGING NETWORK MEDI',
 'CALIFORNIA PEOPLE COUNSELING C',
 'CAROLINA COMMUNITY MENTAL HEAL',
 'CCRN OPERATOR, LLC',
 'CENTRAL KENTUCKY FAMILY PH

In [94]:
leie_org['BUSNAME'].value_counts()

LEHIGH VALLEY COMMUNITY MENTAL    4
LA FUENTE OCULAR PROSTHETICS,     2
NORTHEAST COMMUNITY MENTAL HEA    2
CALIFORNIA PEOPLE COUNSELING C    1
NARCO FREEDOM, INC                1
NEW PLAZA GROUP, INC              1
DESERET HEALTH GROUP, LLC         1
UNICARE AMBULANCE, LLC            1
FRIENDS ADULT DAY CARE, INC       1
A & H VITAMINS SUPPLY CORP        1
NEW DAY PHARMACY                  1
CALIFORIA IMAGING NETWORK MEDI    1
184TH STREET PHARMACY CORP        1
EMPLOYMENT & ASSESSMENT SOLUTI    1
ST JUDE HOME CARE, INC            1
ADEL PHYSICAL THERAPY, PC         1
WILLOWS CONSULTING COMPANY        1
CLIFFORD J HURLEY, D O, LLC       1
TOTAL LAB CARE, LLC               1
AVENUE T CHIROPRACTIC PC          1
ISERVE TECHNOLOGIES, INC          1
LIFELINE HOME HEALTH SERVICES,    1
NORTH AMERICA DRUGS CORP          1
BOLA ELEMUREN, MD PROFESSIONAL    1
BELL GARDENS PHARMACY             1
ASTRAMED PHYSICIAN PC             1
LAZARUS CARE, LLC                 1
MIDWAY DRUGS                

**Convert org name to lower case and remove punctuation**

In [156]:
## Convert Org last name to lower case

leie_org['BUSNAME']= leie_org['BUSNAME'].apply(lambda x: x.lower())

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [157]:
## Remove punctuations from org name using the string module (trial run):

import string

punct = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{}~'   # `|` is not present here
transtab = str.maketrans(dict.fromkeys(punct, ''))

leie_org['BUSNAME'] = '|'.join(leie_org['BUSNAME'].tolist()).translate(transtab).split('|')

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [158]:
leie_org['BUSNAME'].unique().tolist()

['184th street pharmacy corp',
 '716 transportation inc',
 'a  h vitamins supply corp',
 'a  y medical supply inc',
 'a fair deal pharmacy inc',
 'a service cab co inc',
 'aaa plus home health care llc',
 'acacia mental health clinic l',
 'ace clinique of medicine llc',
 'adel physical therapy pc',
 'albert ades m d p a',
 'allerton park pharmacy inc',
 'ambama clinic p c',
 'andas pharmaceutical supply co',
 'aocdme corp',
 'arms of grace humanitarian ser',
 'astramed physician pc',
 'autumn health care of zanesvil',
 'avenue t chiropractic pc',
 'because we care llc',
 'bell gardens pharmacy',
 'bennett surgical supply inc',
 'best care medical management',
 'bestcare laboratory services',
 'biodiagnostic laboratory servi',
 'bola elemuren md professional',
 'bright rx inc',
 'brooklyn comprehensive care me',
 'califoria imaging network medi',
 'california people counseling c',
 'carolina community mental heal',
 'ccrn operator llc',
 'central kentucky family pharma',
 'changing step

**Split the business name string into multiple words**

In [159]:
## Split the business name string into multiple words, and put each into a column

tempbus = leie_org['BUSNAME'].str.split(' ', expand=True).rename(columns = lambda x: "BUSNAME"+str(x+1))

In [160]:
type(tempbus)
tempbus.head()
tempbus.shape

(143, 8)

In [161]:
#Concatenate the different business name strings with the leie_org dataframe.

##leie_org = pd.concat([leie_org, tempbus], axis = 1) ##Works, but messes up the order of columns.

# Unlike pd.concat, the code below preserves the order of columns in leie_org, and puts the split cols at the end.
leie_org = leie_org.join(tempbus)

In [162]:
leie_org.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,EXCLYEAR,zip_len,zip_new,BUSNAME1,BUSNAME2,BUSNAME3,BUSNAME4,BUSNAME5,BUSNAME6,BUSNAME7,BUSNAME8
0,NaN,NaN,,184th street pharmacy corp,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,2018-04-19,0,0,NaN,2018,5,10468,184th,street,pharmacy,corp,None,None,None,None
1,NaN,NaN,,716 transportation inc,OTHER BUSINESS,TRANSPORTATION CO,NaN,0,NaN,540 VINE LANE,BUFFALO,NY,14228,1128a1,2019-06-20,0,0,NaN,2019,5,14228,716,transportation,inc,None,None,None,None,None
2,NaN,NaN,,a h vitamins supply corp,OTHER BUSINESS,PHARMACY,NaN,0,NaN,592 PALISADE AVENUE,TEANECK,NJ,7666,1128a1,2018-04-19,0,0,NaN,2018,4,07666,a,,h,vitamins,supply,corp,None,None
3,NaN,NaN,,a y medical supply inc,DME COMPANY,DME - GENERAL,NaN,1942476080,NaN,"6310 108TH STREET, APT 6J",FOREST HILLS,NY,11375,1128b8,2017-05-18,0,0,NaN,2017,5,11375,a,,y,medical,supply,inc,None,None
4,NaN,NaN,,a fair deal pharmacy inc,OTHER BUSINESS,PHARMACY,NaN,1891731758,NaN,"C/O P O BOX 329014, #69709-05",BROOKLYN,NY,11232,1128b8,2017-05-18,0,0,NaN,2017,5,11232,a,fair,deal,pharmacy,inc,None,None,None


**Work with the address string**

In [102]:
leie_org['ADDRESS'].unique().tolist()

['69 E 184TH ST',
 '540 VINE LANE',
 '592 PALISADE AVENUE',
 '6310 108TH STREET, APT 6J',
 'C/O  P O BOX 329014, #69709-05',
 '3704 AIRLINE DRIVE',
 '6160 104TH CIRCLE NORTH',
 '5228 W FOND DU LAC AVENUE',
 'P O BOX 14500, #17745-032',
 'C/O 8605 60TH ROAD, APT 3K',
 'P O BOX 779800, #67194-050',
 '645 ALLERTON AVENUE',
 '16 HONEYSUCKLE LANE',
 '8015 S US HIGHWAY 75',
 '2821 CRENSHAW BLVD, 100',
 'C/O P O BOX 300, #69885-054',
 '1420 AUTUMN DRIVE',
 '841 KING STREET',
 '6748 LAVENDER LILLY LANE',
 '7625 EASTERN AVE, STE C',
 '21 BARSTOW ROAD, APT 5D',
 '501 W GLENAOKS BLVD',
 '202 N TEXAS AVENUE, STE 100',
 '120 BUNKER HILL ROAD',
 '740 S AMY LANE',
 '183 BUFFALO AVENUE',
 'C/O P O BOX 1000, #83383-053',
 '523 W 5TH STREET, SUITE 626',
 '4928 LANKERSHIM BLVD',
 '4020 CAPITAL BLVD, SUITE 102',
 '128 PHOENIX MILLS ROAD',
 '201 BOSTON SQUARE',
 '14540 HAMLIN ST, STE B',
 '591 ROUTE 216',
 '550 E CHAPMAN AVENUE, STE E',
 'P O BOX 122',
 '24 SPLIT ROCK ROAD',
 '207 5TH AVENUE, N',
 '1025 5T

In [103]:
leie_org['ADDRESS'].value_counts()

207 5TH AVENUE, N                 5
592 PALISADE AVENUE               5
1916 81ST STREET, APT 1           3
727 WICKER AVENUE                 2
45 HALE PL                        2
9050 W TROPICANA AVE, UNIT 115    2
1044 RUNNING BROOK DRIVE          2
333 RECTOR PLACE, APT 706         1
6613 COLUMBUS AVENUE              1
8265 MOUNT MCKINLEY DR            1
210 N 6TH STREET, #214            1
183 BUFFALO AVENUE                1
21859 OLDFIELD AVE, N             1
69 E 184TH ST                     1
C/O 8605 60TH ROAD, APT 3K        1
645 ALLERTON AVENUE               1
540 VINE LANE                     1
4928 LANKERSHIM BLVD              1
550 E CHAPMAN AVENUE, STE E       1
6748 LAVENDER LILLY LANE          1
1704 W MANCHESTER AVE, STE 103    1
1 CHISHOLM TRAIL, STE 300         1
725 N SHEPARD ST                  1
7625 EASTERN AVE, STE C           1
12605 BISCAYNE BAY DRIVE          1
3501 ATLANTIC BLVD                1
4545 W TOUHY AVE, UNIT 516W       1
C/O P O BOX 1000, #83383-053

**Split the address string into two parts at the comma to match street 1 and street 2 of CMS files**

In [125]:
## split the address field on the comma and create a temp new data frame with split value columns 
## Splitting is needed to remove things like Suite and Apartment, and move it to a separate column.

new = leie_org['ADDRESS'].str.split(",", n = 1, expand = True) 
  
# make a seprate var for street 1 from the 'new' df 
leie_org["Street_1"]= new[0] 
  
# make a separate var for street 2 from 'new' df
leie_org["Street_2"]= new[1] 

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [126]:
leie_org.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,EXCLYEAR,zip_len,zip_new,Street_1,Street_2
0,NaN,NaN,,184th street pharmacy corp,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,2018-04-19,0,0,NaN,2018,5,10468,69 E 184TH ST,None
1,NaN,NaN,,716 transportation inc,OTHER BUSINESS,TRANSPORTATION CO,NaN,0,NaN,540 VINE LANE,BUFFALO,NY,14228,1128a1,2019-06-20,0,0,NaN,2019,5,14228,540 VINE LANE,None
2,NaN,NaN,,a h vitamins supply corp,OTHER BUSINESS,PHARMACY,NaN,0,NaN,592 PALISADE AVENUE,TEANECK,NJ,7666,1128a1,2018-04-19,0,0,NaN,2018,4,07666,592 PALISADE AVENUE,None
3,NaN,NaN,,a y medical supply inc,DME COMPANY,DME - GENERAL,NaN,1942476080,NaN,"6310 108TH STREET, APT 6J",FOREST HILLS,NY,11375,1128b8,2017-05-18,0,0,NaN,2017,5,11375,6310 108TH STREET,APT 6J
4,NaN,NaN,,a fair deal pharmacy inc,OTHER BUSINESS,PHARMACY,NaN,1891731758,NaN,"C/O P O BOX 329014, #69709-05",BROOKLYN,NY,11232,1128b8,2017-05-18,0,0,NaN,2017,5,11232,C/O P O BOX 329014,#69709-05


In [127]:
## Convert Street_1 address to lower case

leie_org['Street_1']= leie_org['Street_1'].apply(lambda x: x.lower())

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [128]:
## Remove punctuations etc from Street_1 name. This version uses Regex.

leie_org['Street_1'] = leie_org['Street_1'].str.replace(r'[^\w\s]+', '')

C:\Users\aparn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [129]:
leie_org.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,EXCLYEAR,zip_len,zip_new,Street_1,Street_2
0,NaN,NaN,,184th street pharmacy corp,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,2018-04-19,0,0,NaN,2018,5,10468,69 e 184th st,None
1,NaN,NaN,,716 transportation inc,OTHER BUSINESS,TRANSPORTATION CO,NaN,0,NaN,540 VINE LANE,BUFFALO,NY,14228,1128a1,2019-06-20,0,0,NaN,2019,5,14228,540 vine lane,None
2,NaN,NaN,,a h vitamins supply corp,OTHER BUSINESS,PHARMACY,NaN,0,NaN,592 PALISADE AVENUE,TEANECK,NJ,7666,1128a1,2018-04-19,0,0,NaN,2018,4,07666,592 palisade avenue,None
3,NaN,NaN,,a y medical supply inc,DME COMPANY,DME - GENERAL,NaN,1942476080,NaN,"6310 108TH STREET, APT 6J",FOREST HILLS,NY,11375,1128b8,2017-05-18,0,0,NaN,2017,5,11375,6310 108th street,APT 6J
4,NaN,NaN,,a fair deal pharmacy inc,OTHER BUSINESS,PHARMACY,NaN,1891731758,NaN,"C/O P O BOX 329014, #69709-05",BROOKLYN,NY,11232,1128b8,2017-05-18,0,0,NaN,2017,5,11232,co p o box 329014,#69709-05


**Read in the individual level file from LEIE**

In [95]:
## Subset the individuals to examine them more closely without organziations

leie_ind = leie_m[leie_m['LASTNAME'].notnull()]

In [96]:
type(leie_ind)

pandas.core.frame.DataFrame

In [97]:
leie_ind.shape

(7544, 21)

In [98]:
# Columns with missings

leie_ind.columns[leie_ind.isnull().any()].tolist()

['FIRSTNAME', 'BUSNAME', 'UPIN', 'DOB', 'WVRSTATE']

In [99]:
# Number missings by column

len(leie_ind.index) - leie_ind.count()

LASTNAME         0
FIRSTNAME        1
MIDNAME          0
BUSNAME       7544
GENERAL          0
SPECIALTY        0
UPIN          7396
NPI              0
DOB              1
ADDRESS          0
CITY             0
STATE            0
ZIP              0
EXCLTYPE         0
EXCLDATE         0
REINDATE         0
WAIVERDATE       0
WVRSTATE      7544
EXCLYEAR         0
zip_len          0
zip_new          0
dtype: int64

In [100]:
leie_ind['LASTNAME'].unique().tolist()

['AAMIR',
 'AARON',
 'AASEN',
 'ABALOS',
 'ABARQUEZ',
 'ABBASSI',
 'ABBOTT',
 'ABDELRAHMAN',
 'ABDULLAHI',
 'ABERASTURIA',
 'ABERIN',
 'ABERNATHY',
 'ABIFARIN',
 'ABOUZAHRA',
 'ABRAHAM',
 'ABREU',
 'ACEDO',
 'ACOSTA',
 'ACQUAH',
 'ACREE',
 'ACRI',
 'ADAMES',
 'ADAMIK',
 'ADAMO',
 'ADAMS',
 'ADAMSON',
 'ADAN',
 'ADARMES-VAZQUEZ',
 'ADEBAYO',
 'ADELMAN',
 'ADEM',
 'ADEMIJU',
 'ADEOYE',
 'ADEYOLA',
 'ADKINS',
 'ADOH',
 'ADOLFSON',
 'ADU GYAMFI',
 'AFFRUNTI',
 'AGBONKPOLOR',
 'AGEE',
 'AGGARWAL',
 'AGOMUOH',
 'AGUILA',
 'AGUILAR',
 'AGUILERA ESCALONA',
 'AGUIRRE',
 'AGUZIE',
 'AGYAPONG',
 'AGYEMAN',
 'AHLBECK',
 'AHLGRIMM',
 'AHMAD',
 'AHMADANI',
 'AHMED',
 'AHN',
 'AIDASANI',
 'AIELLO',
 'AIKEN',
 'AIKENS',
 'AINABE',
 'AINSLEY',
 'AIRHART',
 'AJAYI',
 'AJIM',
 'AKAMANTI',
 'AKANDE',
 'AKBAR',
 'AKHARAMEN',
 'AKHTAR',
 'AKINGBADE',
 'AKINOLA',
 'AKINS',
 'AKINSETE',
 'ALAM',
 'ALAMAH',
 'ALANIZ',
 'ALARI',
 'ALBADRY',
 'ALBAYATI',
 'ALBERT',
 'ALDERMAN',
 'ALDRICH',
 'ALEGRIA',
 'ALERT',


In [101]:
leie_ind['LASTNAME'].value_counts()

SMITH          69
JOHNSON        60
BROWN          52
JONES          47
WILLIAMS       43
DAVIS          35
MILLER         33
HARRIS         31
TAYLOR         29
THOMAS         29
JACKSON        28
GONZALEZ       28
MOORE          27
RODRIGUEZ      25
WILSON         24
LOPEZ          23
HERNANDEZ      22
WHITE          20
PEREZ          20
MARTIN         19
GARCIA         19
COLLINS        19
MARTINEZ       19
THOMPSON       18
CLARK          18
YOUNG          18
GRAY           17
BAILEY         17
ALLEN          17
PARKER         16
               ..
WORRELL         1
MONACO          1
FIFIELD         1
KERSTEEN        1
OFOEGBU         1
GARBARINO       1
FALCON          1
FELTON          1
LOMINI          1
WALASZEWSKI     1
SHAPRAN         1
DINH            1
SPADARO         1
KALINOSKI       1
ERHO            1
CORTEZ          1
DUBOR           1
HOPPE           1
WIESLER         1
EDMONDS         1
POLLARD         1
MULAMALLA       1
BUDYNSKI        1
MILLIGAN        1
DANE      